In [9]:
!pip install openai --upgrade

In [10]:
import os
import openai
from openai import OpenAI


In [13]:
openai.api_key = keyChat
client = OpenAI(api_key=keyChat)

In [18]:

def test_openai_api():
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user",
                 "content": "Hello, who won the world series in 2020?"}
            ],
            max_tokens=5
        )
        print("API Call Successful!")
        print(response.choices[0].message['content'])
    except Exception as e:
        print(f"Error during API call: {e}")

test_openai_api()


API Call Successful!
Error during API call: 'ChatCompletionMessage' object is not subscriptable


In [10]:
pdf_paths = [
    '/content/ATTACK_Design_and_Philosophy_March_2020.pdf',
    '/content/CELEX_32016R0679_EN_TXT.pdf',
    '/content/diamond.pdf',
    '/content/nist.sp.800-150.pdf'
]
urls = [
    'https://oasis-open.github.io/cti-documentation/',
    'https://capec.mitre.org'
]


In [12]:
import csv
import pdfplumber
import requests
from bs4 import BeautifulSoup
import openai
import os

# Set your OpenAI API key (ensure to replace 'your-api-key-here' with your actual API key)
openai.api_key = os.getenv("OPENAI_API_KEY",'' )

print(os.getenv("OPENAI_API_KEY"))


def extract_text_from_pdfs(pdf_paths):
    all_text = ""
    for pdf_path in pdf_paths:
        try:
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    text = page.extract_text()
                    if text:
                        all_text += text + "\n"
        except Exception as e:
            print(f"Error extracting text from {pdf_path}: {e}")
    return all_text

def extract_text_from_urls(urls):
    all_text = ""
    for url in urls:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                all_text += soup.get_text() + "\n"
            else:
                print(f"Failed to retrieve {url}: HTTP {response.status_code}")
        except Exception as e:
            print(f"Error retrieving {url}: {e}")
    return all_text

def generate_question(chunk, max_tokens=150):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a cybersecurity expert specializing in cyber threat intelligence."},
                {"role": "user", "content": f"Given the text below, please generate a single multiple-choice question with four options based on the following requirements:\n"
                                            "1. Question Format: The question must have four options. The options should be challenging and require careful consideration. Avoid creating options that could be interpreted as correct under different circumstances.\n"
                                            "2. Target Audience: The question should be suitable for security professionals with three to five years of experience in cyber threat intelligence. Avoid generic questions such as “What is the objective?”, “Which operating system can be targeted?”.\n"
                                            "3. Content Coverage: Aim to cover various sections of the document to ensure a comprehensive evaluation of the candidate’s knowledge. Include context-specific questions that require an understanding of the document’s content.\n"
                                            "4. Technical Precision: Use precise terminology and concepts relevant to cyber threat intelligence. Incorporate situational or scenario-based questions where applicable.\n"
                                            "5. Include Technique IDs and Names: Ensure that the question, where applicable, mentions both the ID and the full name of the MITRE ATT&CK pattern technique.\n"
                                            "6. Premise Inclusion: The question should include a premise indicating it pertains to MITRE ATT&CK, specifying the relevant platform (Enterprise, ICS, or Mobile) where necessary.\n"
                                            "7. Output Format: Return the output in TSV format (must be tab-separated) with the following columns: Question, Option A, Option B, Option C, Option D, Correct Answer (A, B, C, D), and Explanation.\n"
                                            "Important: Only return the TSV content as specified. Do not include any additional text or commentary outside the TSV format.\n"
                                            "Text:\n\n{chunk}"}
            ],
            max_tokens=max_tokens,
            temperature=0.7
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        print(f"Error generating question: {e}")
        return ""

def process_and_generate_questions(pdf_paths, urls, output_csv):
    # Extract text from PDFs and URLs
    pdf_text = extract_text_from_pdfs(pdf_paths)
    url_text = extract_text_from_urls(urls)
    combined_text = pdf_text + url_text

    chunks = [combined_text[i:i+2000] for i in range(0, len(combined_text), 2000)]

    # Generate questions and save to CSV
    question_count = 0
    with open(output_csv, mode='w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Correct Answer (A, B, C, D)', 'Explanation']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter='\t')

        writer.writeheader()

        for chunk in chunks:
            tsv_content = generate_question(chunk)
            if tsv_content:
                # Split TSV content into rows and write each row to the CSV
                rows = tsv_content.split('\n')
                for row in rows:
                    columns = row.split('\t')
                    if len(columns) == 7:  # Ensure we have the correct number of columns
                        writer.writerow({
                            'Question': columns[0],
                            'Option A': columns[1],
                            'Option B': columns[2],
                            'Option C': columns[3],
                            'Option D': columns[4],
                            'Correct Answer (A, B, C, D)': columns[5],
                            'Explanation': columns[6]
                        })
                        question_count += 1

    print(f"Number of questions generated: {question_count}")


output_csv = "generated_questions.tsv"
process_and_generate_questions(pdf_paths, urls, output_csv)


None
Error generating question: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

Error generating question: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

Error g